# 제주도 핫플레이스 - 인스타그램 크롤링

In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
from urllib.parse import quote

In [8]:
chromedriver = 'C:/Users/imkyu/Downloads/chromedriver.exe'
driver = webdriver.Chrome(chromedriver)

### 1. Instagram 접속하여 로그인

In [9]:
insta_url = 'https://www.instagram.com'
driver.get(insta_url)
time.sleep(1)

In [13]:
with open('password.txt') as f:
    password = f.read()

In [14]:
email = 'llibeauty'
input_email = driver.find_element_by_css_selector('._2hvTZ.pexuQ.zyHYP')
input_email.clear()
input_email.send_keys(email)

In [15]:
input_pwd = driver.find_element_by_name('password')
input_pwd.clear()
input_pwd.send_keys(password)
input_pwd.submit()
time.sleep(1)

In [16]:
# 팝업-알림(로그인정보 저장-안함)
try:
    driver.find_element_by_css_selector('.sqdOP.yWX7d.y3zKF').click()
    time.sleep(1)
except:
    pass

In [17]:
# 팝업 - 알림(나중에 하기)
try:
    driver.find_element_by_css_selector('.aOOlW.HoLwm').click()
    time.sleep(1)
except:
    pass

### 2. 제주도 맛집 으로 검색

In [18]:
keyword = '제주도맛집'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
url

'https://www.instagram.com/explore/tags/%EC%A0%9C%EC%A3%BC%EB%8F%84%EB%A7%9B%EC%A7%91'

In [19]:
driver.get(url)
time.sleep(5)   #시간 충분히 줘야 함(화면전환될떄까지기달)

### 3. 첫번째 게시글 열기

In [36]:
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

### 4. 게시글 정보 가져오기

In [37]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [38]:
# 1)본문내용 가져오기
import unicodedata
try:
    content = soup.select_one('div.C4VMK > span').text
    content = unicodedata.normalize('NFC', content)
except:
    content = ' '
content

'42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐히 들어간 순두부 찌개다. #신해바라기분식 #제주도맛집뜨끈한 국물과 두부 한 숟갈을 떠먹으면 화끈한 매운맛과 뜨거운 열기가 식욕을 자극한다. (신라면보다 조금 매운 정도?)맵기 조절이 가능하기에 취향에 따라 먹을 수 있다. 매운 걸 먹지 못하는 일행이 있다면, 백순두부도 좋다. 더 담백하고 맑은 국물로 두부맛에 집중할 수 있다. 보글보글 끓어나오는 콩비지 느낌의 순두부 찌개와 오징어 젓갈이 기억에 남는 제주도 로컬 맛집이다. 📍 제주 일도1동 1342-4◾순두부찌개 ₩7,000🏷 피웨 부캐 노포 간판 수집가 @nopo__joa🏷 미식가들의 맛집 모음집 @e.a.t.club#순두부찌개 #국밥맛집 #제주맛집 #제주시맛집 #피그웨이브_제주'

In [40]:
# 2)본문 내용에서 해시태그만 가져오기(정규표현식: '#[^\s#,\\]+')
import re
tags = re.findall(r'#[^\s#,\\]+', content)
tags

['#신해바라기분식', '#제주도맛집뜨끈한', '#순두부찌개', '#국밥맛집', '#제주맛집', '#제주시맛집', '#피그웨이브_제주']

In [41]:
# 3)작성일자 정보 가져오기
date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
date

'2021-08-18'

In [42]:
# 4) 좋아요 수 가져오기: 좋아요 1,296개
try:
    like = soup.select_one('div.Nm9Fw').text[4:-1]
except:
    like = 0
like

'1,247'

In [26]:
# 5) 위치정보 가져오기
try:
    place = soup.select_one('div.M30cS')[0].text
    place = unicodedata.normalize('NFC', place)
except:
    place = ''
place

''

In [43]:
row = [content, tags, date, like, place]
row

['42년 전통 도민 맛집. 태양을 품은 듯 화끈한 비주얼이다. 포슬포슬한 두부가 푸짐히 들어간 순두부 찌개다. #신해바라기분식 #제주도맛집뜨끈한 국물과 두부 한 숟갈을 떠먹으면 화끈한 매운맛과 뜨거운 열기가 식욕을 자극한다. (신라면보다 조금 매운 정도?)맵기 조절이 가능하기에 취향에 따라 먹을 수 있다. 매운 걸 먹지 못하는 일행이 있다면, 백순두부도 좋다. 더 담백하고 맑은 국물로 두부맛에 집중할 수 있다. 보글보글 끓어나오는 콩비지 느낌의 순두부 찌개와 오징어 젓갈이 기억에 남는 제주도 로컬 맛집이다. 📍 제주 일도1동 1342-4◾순두부찌개 ₩7,000🏷 피웨 부캐 노포 간판 수집가 @nopo__joa🏷 미식가들의 맛집 모음집 @e.a.t.club#순두부찌개 #국밥맛집 #제주맛집 #제주시맛집 #피그웨이브_제주',
 ['#신해바라기분식', '#제주도맛집뜨끈한', '#순두부찌개', '#국밥맛집', '#제주맛집', '#제주시맛집', '#피그웨이브_제주'],
 '2021-08-18',
 '1,247',
 '']

### 5. 다음 게시글로 이동

In [44]:
driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
time.sleep(2)

In [45]:
# 6을 하기 위해 팝업 화면을 없앰(즉, X를 클릭)
driver.find_element_by_xpath('/html/body/div[6]/div[3]/button').click()
time.sleep(2)

### 6. 여러 개의 게시글 가져오기

In [69]:
from tqdm.notebook import tqdm

In [76]:
def get_content(driver):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    try:
        content = soup.select_one('div.C4VMK > span').text
        content = unicodedata.normalize('NFC', content)
    except:
        content = ' '
    
    tags = re.findall(r'#[^\s#,\\]+', content)
    date = soup.select_one('time.FH9sR.Nzb55')['datetime'][:10]
    
    try:
        like = soup.select_one('div.Nm9Fw').text[4:-1]
    except:
        like = 0

    try:
        place = soup.select_one('div.M30cS')[0].text
        place = unicodedata.normalize('NFC', place)
    except:
        place = ''
    return [content, tags, date, like, place]

### - 제주도맛집, 제주맛집, 제주도관광, 제주여행

In [77]:
keyword = '제주여행'
search_url = 'https://www.instagram.com/explore/tags/'
url = f'{search_url}{quote(keyword)}'
driver.get(url)
time.sleep(5)
driver.find_element_by_css_selector('div._9AhH0').click()
time.sleep(1)

In [78]:
#200개 정도가 한계
results = []
for _ in tqdm(range(200)):
    try:
        row = get_content(driver)
        results.append(row)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)
    except:
        time.sleep(3)
        driver.find_element_by_css_selector('a._65Bje.coreSpriteRightPaginationArrow').click()
        time.sleep(2)

  0%|          | 0/200 [00:00<?, ?it/s]

In [79]:
df = pd.DataFrame(results, columns=['content','tags', 'date','like','place'])
df.head()

,content,tags,date,like,place
0,.빵수니 빵도리들을 위한 <제주 빵지순례 모음! >맛있고 가볼만한 빵집들을 모아봤어...,"[#제주맛집, #제주도맛집, #제주공항근처맛집, #제주시맛집, #애월맛집, #중문맛...",2021-08-19,"1,217",
1,[1박에 10만원 제주도 풀빌라 대공개!😍]제주도 6시 방향에 있는 '르페도라' 풀...,[#동행스타그램],2021-08-19,305,
2,하루종일 파란 날 요즘 내 짝꿍 오코백이랑 .. 🧵🪡🤍🤍🤍 (feat 오트에코백),[],2021-08-18,"2,540",
3,가로사진이 훨씬 더 어울리는 공간이라 이번 피드는 좀🤔일찍 가면 홀로 명상(Mind...,"[#제주신상카페, #월정리카페, #아이엠더그라스]",2021-08-18,920,
4,@uniqcong_official #유니콩💚👖,[#유니콩💚👖],2021-08-19,"1,306",


In [80]:
df.tail()

,content,tags,date,like,place
194,초록초록한 세상에서 즐거운 시간 가득 가졌어요.책 좋아하는 둘째아이는 도서관에서 나...,"[#제주돌하르방미술관, #제주아이와가볼만한곳......, #제주여행코스, #제주동쪽...",2021-08-19,1,
195,제주 둘째날한담해변을 지나 용머리해안숙소로 돌아오는길에 신풍해안도로근처 카페뷰가 너...,"[#용머리해안, #뷰맛집, #신풍해안도로, #풍차, #메밀꽃, #제주여행, #여행스...",2021-08-19,8,
196,첫날 일정 마무리는 역시 고기!👍#제주#제주여행#혼여행#현지인맛집#아라돈사촌#존맛탱👍,"[#제주, #제주여행, #혼여행, #현지인맛집, #아라돈사촌, #존맛탱👍]",2021-08-19,5,
197,.미친 하늘과 구름.적당히 날리는 바람에 머리카락과 하늘하늘거리는 원피스.이거 한 ...,"[#여행은, #사진이지사진작가.]",2021-08-19,fafa님 외 16명이 좋아합니,
198,그냥 인터넷에 나온 밥집 찾아간건데 가게 내부에서 백종원 아저씨의 사진을 보았을 때...,"[#피자하나잘하는집, #맛집이네, #돈까스맛집, #피자맛집, #보스코화덕피자, #제...",2021-08-19,18,


In [81]:
df.to_csv('Data/제주여행.csv', index=False)

In [82]:
driver.close()

- 4가지 CSV 파일의 중복을 제거한 후 통합 저장